# LangChain's components

## Models and Prompts

In [1]:
from langchain_community.llms.gpt4all import GPT4All
# Make sure the model path is correct for your system!

MODEL_PATH = r'C:\Users\jsshe\.cache\gpt4all\Meta-Llama-3-8B-Instruct.Q4_0.gguf'

llm = GPT4All(
    model=MODEL_PATH,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
    seed=42,
)

In [2]:
from langchain import PromptTemplate

template = """Sentence: {sentence}
Translation in {language}:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

print(prompt.format(sentence = "the cat is on the table", language = "spanish"))

Sentence: the cat is on the table
Translation in spanish:


In [3]:
print(llm(prompt.format(sentence = "the cat is on the table", language = "spanish")))

c:\Users\jsshe\Documents\learning\oreilly\Building-LLM-Powered-Applications\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 el gato está en la mesa
In this sentence, "the" is an article that refers to a specific noun (cat) and indicates its location ("on the table"). The verb "is" links the subject (cat) with the predicate (on the table). This sentence follows the typical structure of English sentences: Subject-Verb-Predicate.
The translation in Spanish maintains this same structure, using the article "el" to refer to a specific noun (gato), and indicating its location ("en la mesa"). The verb "está" links the subject (gato) with the predicate (en la mesa). This sentence follows the typical structure of Spanish sentences: Subject-Verb-Predicate.
In both languages, this sentence is simple because it only has one clause. It's a declarative sentence that states a fact or situation. The meaning and function are similar in English and Spanish, but there may be some differences in nuance depending on cultural context and the specific words used. For example, "on the table" could imply a sense of messiness or dis

## Data Connections

### Document loaders

In [4]:
import csv

# Sample data
data = [
    ['Name', 'Age', 'City'],
    ['John', 25, 'New York'],
    ['Emily', 28, 'Los Angeles'],
    ['Michael', 22, 'Chicago']
]

# File name
file_name = 'sample.csv'

# Write data to CSV file
with open(file_name, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

print(f'Sample CSV file "{file_name}" generated and saved.')



Sample CSV file "sample.csv" generated and saved.


In [5]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='sample.csv')
data = loader.load()
print(data)

[Document(metadata={'source': 'sample.csv', 'row': 0}, page_content='Name: John\nAge: 25\nCity: New York'), Document(metadata={'source': 'sample.csv', 'row': 1}, page_content='Name: Emily\nAge: 28\nCity: Los Angeles'), Document(metadata={'source': 'sample.csv', 'row': 2}, page_content='Name: Michael\nAge: 22\nCity: Chicago')]


### Document splitters

In [6]:

# Sample sentences about mountains and nature
content = """Amidst the serene landscape, towering mountains stand as majestic guardians of nature's beauty.
The crisp mountain air carries whispers of tranquility, while the rustling leaves compose a symphony of wilderness.
Nature's palette paints the mountains with hues of green and brown, creating an awe-inspiring sight to behold.
As the sun rises, it casts a golden glow on the mountain peaks, illuminating a world untouched and wild."""

# File name
file_name = 'mountain.txt'

# Write content to text file
with open(file_name, 'w') as txtfile:
    txtfile.write(content)

#print(f'Sample text file "{file_name}" generated and saved.')


with open('mountain.txt') as f:
    mountain = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(

    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len
)

texts = text_splitter.create_documents([mountain])
print(texts[0])
print(texts[1])
print(texts[2])

page_content='Amidst the serene landscape, towering mountains stand as majestic guardians of nature's beauty.'
page_content='The crisp mountain air carries whispers of tranquility, while the rustling leaves compose a'
page_content='leaves compose a symphony of wilderness.'


### Text embedding models

In [7]:
from nomic import embed

embeddings_output = embed.text([
    "Good morning!",
    "Oh, hello!",
    "I want to report an accident",
    "Sorry to hear that. May I ask your name?",
    "Sure, Mario Rossi."
    ],
    model='nomic-embed-text-v1.5',
    task_type='search_document',
    inference_mode='local',
)

embeddings = embeddings_output['embeddings']

print("Embed documents:")
print(f"Number of vector: {len(embeddings)}; Dimension of each vector: {len(embeddings[0])}")

embedded_query_output = embed.text(
    ['What was the name mentioned in the conversation?'],
    model='nomic-embed-text-v1.5',
    task_type='search_query',
    inference_mode='local',
    )

embedded_query = embedded_query_output['embeddings'][0]

print("Embed query:")
print(f"Dimension of the vector: {len(embedded_query)}")
print(f"Sample of the first 5 elements of the vector: {embedded_query[:5]}")


Embed documents:
Number of vector: 5; Dimension of each vector: 768
Embed query:
Dimension of the vector: 768
Sample of the first 5 elements of the vector: [0.03053959831595421, 0.029206879436969757, -0.16909408569335938, -0.07121364772319794, 0.04683174565434456]


In [8]:
#saving the conversation in a txt file
# List of dialogue lines
dialogue_lines = [
    "Good morning!",
    "Oh, hello!",
    "I want to report an accident",
    "Sorry to hear that. May I ask your name?",
    "Sure, Mario Rossi."
]

# File name
file_name = 'dialogue.txt'

# Write dialogue lines to text file
with open(file_name, 'w') as txtfile:
    for line in dialogue_lines:
        txtfile.write(line + '\n')

print(f'Dialogue text file "{file_name}" generated and saved.')


Dialogue text file "dialogue.txt" generated and saved.


### Vector stores

In [41]:
from langchain_core.embeddings import Embeddings
class NomicEmbedModel(Embeddings):
    def __init__(
        self,
        model='nomic-embed-text-v1.5',
        inference_mode='local',
        ) -> None:
        self.model = model
        self.inference_mode = inference_mode
    
    def embed_documents(self, text):
        embeddings_output = embed.text(
            text,
            model= self.model,
            task_type='search_document',
            inference_mode=self.inference_mode,
            )
        return embeddings_output['embeddings']
    
    def embed_query(self, text):
        embeddings_output = embed.text(
            text,
            model=self.model,
            task_type='search_query',
            inference_mode=self.inference_mode,
            )
        return embeddings_output['embeddings'][0]

In [43]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS


# Load the document, split it into chunks, embed each chunk and load it into the vector store.

raw_documents = TextLoader('dialogue.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator = "\n",)
documents = text_splitter.split_documents(raw_documents)
lines = [doc.page_content for doc in documents]
embeder = NomicEmbedModel()
db = FAISS.from_documents(documents, embeder)

In [45]:
query = "What is the reason for calling?"
embedded_query = embeder.embed_query([query])
docs = db.similarity_search(embedded_query)
# print(docs[0].page_content)

AttributeError: 'float' object has no attribute 'encode'

In [66]:
print(documents[2])

page_content='I want to report an accident' metadata={'source': 'dialogue.txt'}


### Retrievers

In [75]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

retriever = db.as_retriever()

In [76]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

query = "What was the reason of the call?"
qa.run(query)

' The reason for the call was to report an accident.'

## Memory

In [82]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.llms import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "hi, I'm looking for some ideas to write an essay in AI"}, {"output": "hello, what about writing on LLMs?"})

memory.load_memory_variables({})

{'history': '\nThe human asked for ideas to write an essay in AI and the AI suggested writing on LLMs.'}

In [81]:
ConversationSummaryMemory.save_context?

Signature:
ConversationSummaryMemory.save_context(
    self,
    inputs: 'Dict[str, Any]',
    outputs: 'Dict[str, str]',
) -> 'None'
Docstring: Save context from this conversation to buffer.
File:      c:\users\vaalt\appdata\local\anaconda3\lib\site-packages\langchain\memory\summary.py
Type:      function

## Chains

### Simple Chain

In [84]:
from langchain import PromptTemplate, OpenAI, LLMChain

template = """Sentence: {sentence}
Translation in {language}:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

llm = OpenAI(temperature=0)

llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain.predict(sentence="the cat is on the table", language="spanish")

' El gato está en la mesa.'

### Router chain

In [92]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

llm = OpenAI()

itinerary_template = """You are a vacation itinerary assistant. \
You help customers finding the best destinations and itinerary. \
You help customer screating an optimized itinerary based on their preferences.

Here is a question:
{input}"""

restaurant_template = """You are a restaurant booking assitant. \
You check with customers number of guests and food preferences. \
You pay attention whether there are special conditions to take into account.

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "itinerary",
        "description": "Good for creating itinerary",
        "prompt_template": itinerary_template,
    },
    {
        "name": "restaurant",
        "description": "Good for help customers booking at restaurant",
        "prompt_template": restaurant_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)



In [94]:
print(chain.run("I'm planning a trip from Milan to Venice by car. What can I visit in between?"))



> Entering new MultiPromptChain chain...
itinerary: {'input': "I'm planning a trip from Milan to Venice by car. What attractions can I visit in between?"}
> Finished chain.


Answer: 
There are many attractions that you can visit while traveling from Milan to Venice by car. Some of the most popular attractions include Lake Como, Verona, the Dolomites, and the picturesque towns of Bergamo and Brescia. You can also visit the stunning UNESCO World Heritage Sites in Mantua and Ferrara. Additionally, you can explore some of the local wineries and sample some of the wines of the region.


In [93]:
print(chain.run("I want to book a table for tonight"))



> Entering new MultiPromptChain chain...
restaurant: {'input': 'I want to book a table for tonight'}
> Finished chain.
. How many people are in your party?

Hi there! How many people are in your party for tonight's reservation?


### Sequential Chain

In [3]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# This is an LLMChain to write a synopsis given a title of a play.
llm = OpenAI(temperature=0)
template = """You are a comedian. Generate a joke on the following {topic}
Joke:"""
prompt_template = PromptTemplate(input_variables=["topic"], template=template)
joke_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """You are translator. Given a text input, translate it to {language}
Translation:"""
prompt_template = PromptTemplate(input_variables=["language"], template=template)
translator_chain = LLMChain(llm=llm, prompt=prompt_template)

In [4]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[joke_chain, translator_chain], verbose=True)
translated_joke = overall_chain.run("Cats and Dogs")



> Entering new SimpleSequentialChain chain...

Why did the cat cross the road? To prove to the dog that it could be done!
 ¿Por qué cruzó el gato la carretera? ¡Para demostrarle al perro que se podía hacer!

> Finished chain.


### Transformation chain

In [22]:
# Import the string module
import string

# Define the function
def rename_cat(inputs: dict) -> dict:
  # Open the file in read mode
  text = inputs["text"]
  # Create a table that maps punctuation characters to None
  new_text = text.replace('cat', 'Silvester the Cat')
  # Apply the table to the text and return the result
  return {"output_text": new_text}




In [23]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

with open("Cats&Dogs.txt") as f:
    cats_and_dogs = f.read()


import string



transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=rename_cat
)

template = """Summarize this text:

{output_text}

Summary:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)

sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

sequential_chain.run(cats_and_dogs)

" Silvester the Cat and a dog lived together but did not get along. Silvester the Cat played a prank on the dog which made him angry. When their owner found them fighting, she scolded them and made them apologize. After that, they became friends and learned to respect each other's differences and appreciate each other's strengths."

## Agents

In [5]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["SERPAPI_API_KEY"]

search = SerpAPIWrapper()

In [6]:
tools = [Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events"
    )]

agent = initialize_agent(tools, llm = OpenAI(), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("When was Avatar 2 released?")



> Entering new AgentExecutor chain...
 I need to find out when Avatar 2 was released.
Action: Search
Action Input: "Avatar 2 release date"
Observation: December 16, 2022
Thought: I now know the final answer.
Final Answer: Avatar 2 was released on December 16, 2022.

> Finished chain.


'Avatar 2 was released on December 16, 2022.'

# Start working with LLMs in Hugging Face Hub

In [ ]:
#!pip install python-dotenv   #installing the required package
#!pip install huggingface_hub

#option 1: get your tokens from the .env file

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"]


In [1]:
#option 2: get the token with the getpass function

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()
HUGGINGFACEHUB_API_TOKEN

In [2]:
from langchain import PromptTemplate, LLMChain
from langchain import HuggingFaceHub
question = "What was the first Disney movie?"

template = """Question: {question}

Answer: give a direct answer"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [9]:
repo_id = "tiiuae/falcon-7b-instruct"  
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 1000}
)
print(llm("what was the first disney movie?"))


The first Disney movie was 'Snow White and the Seven Dwarfs', released in 193
